# Adverserial Search 

Two-player, zero-sum, perfect information game:

* Both players have all information
* Gain in position for one player is loss for another

Can play simple games perfectly. Can play complex games beyond ability of any human.

## Chapter 8

## Base Classes for Game Board

A move is defined as an integer. The board keeps track of the turn, the move, the legal moves, and if the game is complete - win or draw. It also has the ability to evaluate - determine the strength of a position.

In [1]:
from __future__ import annotations
from typing import NewType, List
from abc import ABC, abstractmethod

Move = NewType('Move', int)

class Piece:
    """
    Abstract base class for a game piece:
    
    * opposite
    """
    @property
    def opposite(self) -> Piece:
        raise NotImplementedError("Should be implemented by subclasses.")


class Board(ABC):
    """
    Base class for a game board:
    
    * turn
    * move
    * legal_moves
    * win
    * draw
    * evaluate
    
    """
    @property
    @abstractmethod
    def turn(self) -> Piece:
        ...

    @abstractmethod
    def move(self, location: Move) -> Board:
        ...

    @property
    @abstractmethod
    def legal_moves(self) -> List[Move]:
        ...

    @property
    @abstractmethod
    def is_win(self) -> bool:
        ...

    @property
    def is_draw(self) -> bool:
        return (not self.is_win) and (len(self.legal_moves) == 0)

    @abstractmethod
    def evaluate(self, player: Piece) -> float:
        ...
        

Must answer the questions:

* Whose turn is it?
* What legal moves can be played?
* Is the game won?
* Is the game drawn?

If there are no moves and the game is not won, then it is a draw. 

### Actions

* Make a move from current to new position
* Evaluate position to see which player has the advantage

## Tic-Tac Toe Solver

Use the Mini-max algorithm to play the game perfectly. 

Each square is represented as a piece with three options.

In [3]:
from __future__ import annotations
from typing import List
from enum import Enum

class TTTPiece(Piece, Enum):
    """
    Tic-tac-toe piece. Opposite of X == 0. Opposite of E == E.
    
    """
    X = "X"
    O = "O"
    E = " " # stand-in for empty

    @property
    def opposite(self) -> TTTPiece:
        if self == TTTPiece.X:
            return TTTPiece.O
        elif self == TTTPiece.O:
            return TTTPiece.X
        else:
            return TTTPiece.E

def __str__(self) -> str:
    return self.value


In Python abstract base classes, there is no way to specify a subclass must have a particular instance variable, but we can specify a subclass must have a property (which is a function that is called like an attribute). This explains the use of the `turn` property (`@property`) on the tic-tac-toe board.

In [30]:
class TTTBoard(Board):
    """
    Board for tic-tac-toe. Spaces are represented as integers
    
    0 | 1 | 2
    3 | 4 | 5
    6 | 7 | 8
    
    
    """

    def __init__(
        self, position: List[TTTPiece] = [TTTPiece.E] * 9, turn: TTTPiece = TTTPiece.X
    ) -> None:
        self.position: List[TTTPiece] = position
        self._turn: TTTPiece = turn

    @property
    def turn(self) -> Piece:
        return self._turn

    def move(self, location: Move) -> Board:
        """
        Move a position without altering the state of the board. We do not alter the board
        until a move is selected.
        """
        temp_position: List[TTTPiece] = self.position.copy()
        temp_position[location] = self._turn
        return TTTBoard(temp_position, self._turn.opposite)

    @property
    def legal_moves(self) -> List[Move]:
        """
        Find the legal moves on the board.
        """
        return [
            Move(l) for l in range(len(self.position)) if self.position[l] == TTTPiece.E
        ]

    @property
    def is_win(self) -> bool:
        """
        Determine if a player has won the game.
        """
        # three row, three column, and then two diagonal checks
        return (
            self.position[0] == self.position[1]
            and self.position[0] == self.position[2]
            and self.position[0] != TTTPiece.E
            or self.position[3] == self.position[4]
            and self.position[3] == self.position[5]
            and self.position[3] != TTTPiece.E
            or self.position[6] == self.position[7]
            and self.position[6] == self.position[8]
            and self.position[6] != TTTPiece.E
            or self.position[0] == self.position[3]
            and self.position[0] == self.position[6]
            and self.position[0] != TTTPiece.E
            or self.position[1] == self.position[4]
            and self.position[1] == self.position[7]
            and self.position[1] != TTTPiece.E
            or self.position[2] == self.position[5]
            and self.position[2] == self.position[8]
            and self.position[2] != TTTPiece.E
            or self.position[0] == self.position[4]
            and self.position[0] == self.position[8]
            and self.position[0] != TTTPiece.E
            or self.position[2] == self.position[4]
            and self.position[2] == self.position[6]
            and self.position[2] != TTTPiece.E
        )
    
    def evaluate(self, player: Piece) -> float:
        """
        Evaluate a player's position. 
        """
        if self.is_win and self.turn == player:
            return -1
        elif self.is_win and self.turn != player:
            return 1
        else:
            return 0

    def __repr__(self) -> str:
        """
        Representation of the board for convenience.
        """
        return f"""{self.position[0]}|{self.position[1]}|{self.position[2]} 
{self.position[3]}|{self.position[4]}|{self.position[5]}
{self.position[6]}|{self.position[7]}|{self.position[8]}"""

We can search every possible position in the game tree so the `evaluate` method does not need to be approximate. Instead, we can search through the possible outcomes and determine if a player will win along that tree and return the appropriate number in `evaluate` for the move.

## Minimax Algorithm

A method for finding the optimal move in a two-player, zero-sum game with perfect information. The basic idea is for each move:

1. Play out all possible future states of the move in a decision tree.
2. Bubble up the point value associated with the base cases (leaves) of the decision tree. 
3. Choose the move with the highest expected return to the player.
4. Repeat each turn until game is complete.

With a game such as tic-tac-toe, we can play out the entire game from each move and then bubble up the exact point values. With a larger search space, we would have to stop the search at a maximal ply and then approximate the value of all moves.

In [31]:
def minimax(
    board: Board, maximizing: bool, original_player: Piece, max_depth: int = 8
) -> float:
    """
    Play out MiniMax algorithm with a game board for a maximum depth of play (decision tree).
    Returns the best possible evaluation (not move) from a given state. 
    """
    # Base case for recursion
    if board.is_win or board.is_draw or max_depth == 0:
        # Evaluate the completion of the game for the original player
        return board.evaluate(original_player)

    if maximizing:
        # Start with minimal possible so all moves appear better
        best_eval: float = float("-inf")
        for move in board.legal_moves:
            # Recursively play out moves
            result: float = minimax(
                board.move(move),
                maximizing=False,
                original_player=original_player,
                max_depth=max_depth - 1,
            )
            # Determine if move resulted in better evaluate than the current best
            best_eval: float = max(result, best_eval)
        return best_eval
    else:
        # Start with highest possible so all moves appear better
        best_eval: float = float("inf")
        for move in board.legal_moves:
            # Recursively play out moves
            result: float = minimax(
                board.move(move),
                maximizing=True,
                original_player=original_player,
                max_depth=max_depth - 1,
            )
            best_eval: float = min(result, best_eval)
        return best_eval
    
    

In [132]:
from typing import Callable

def find_best_move(board: Board, max_depth: int = 8, algorithm: Callable = minimax) -> Move:
    """
    Find the best possible move from a given state by evaluating all possible moves 
    with the MiniMax algorithm. Returns the best possible move from a given state. 
    """
    best_eval: float = float("-inf")
    best_move: Move = Move(-1)
    
    # Evaluate all possible moves from the current board state
    for move in board.legal_moves:
        result: float = algorithm(
            # Potential move
            board=board.move(move),
            # Now switching to minimizing
            maximizing=False,
            # The original player is the current turn on the board
            original_player=board.turn,
            # Search max_depth levels in the decision tree
            max_depth=max_depth,
        )
        # Record better move if found
        if result > best_eval:
            best_eval = result
            best_move = move
    return best_move

## Unit Testing for Tic-Tac-Toe Mimimax Algorithm

Implement 3 different unit tests to make sure our algorithm chooses the correct position. Even humans can evaluate all potential outcomes from tic-tac-toe to determine optimal move from any board state.

In [133]:
import unittest


class TTTMinimaxTestCase(unittest.TestCase):
    def test_easy_position(self):
        """
        Test that the algorithm can win in one move.
        """
        to_win_easy_position: List[TTTPiece] = [
            TTTPiece.O, TTTPiece.X, TTTPiece.E,
            TTTPiece.X, TTTPiece.O, TTTPiece.X,
            TTTPiece.X, TTTPiece.E, TTTPiece.E,
        ]
        test_board: TTTBoard = TTTBoard(position=to_win_easy_position, turn=TTTPiece.O)
        answer: Move = find_best_move(board=test_board, max_depth=8)
        assert answer == Move(8)

    def test_block_position(self):
        """
        Test that the algorithm can play a blocking move to prevent other player from winning.
        """
        to_block_easy_position: List[TTTPiece] = [
            TTTPiece.E, TTTPiece.X, TTTPiece.E,
            TTTPiece.X, TTTPiece.O, TTTPiece.O,
            TTTPiece.X, TTTPiece.E, TTTPiece.E,
        ]
        test_board: TTTBoard = TTTBoard(
            position=to_block_easy_position, turn=TTTPiece.O
        )
        answer: Move = find_best_move(board=test_board, max_depth=8)
        assert answer == Move(0)
        
    def test_medium_position(self):
        """
        Test that the algorithm can win two moves out.
        """
        to_win_first_position: List[TTTPiece] = [
            TTTPiece.E, TTTPiece.X, TTTPiece.E,
            TTTPiece.X, TTTPiece.O, TTTPiece.E,
            TTTPiece.E, TTTPiece.E, TTTPiece.E,
        ]
        test_board: TTTBoard = TTTBoard(
            position=to_win_first_position, turn=TTTPiece.O
        )
        answer: Move = find_best_move(board=test_board, max_depth=8)
        assert answer == Move(0)
        
        # Check that X blocks to prevent O from winning
        to_win_second_position: List[TTTPiece] = [
            TTTPiece.O, TTTPiece.X, TTTPiece.E,
            TTTPiece.X, TTTPiece.O, TTTPiece.E,
            TTTPiece.E, TTTPiece.E, TTTPiece.E,
        ]
        test_board: TTTBoard = TTTBoard(
        position=to_win_second_position, turn=TTTPiece.X)
        answer: Move = find_best_move(board=test_board, max_depth=8)
        assert answer == Move(8)
        
        # Suppose X made the wrong move, O must now win
        to_win_third_position: List[TTTPiece] = [
            TTTPiece.O, TTTPiece.X, TTTPiece.E,
            TTTPiece.X, TTTPiece.O, TTTPiece.X,
            TTTPiece.E, TTTPiece.E, TTTPiece.E
        ]
        test_board: TTTBoard = TTTBoard(
        position=to_win_second_position, turn=TTTPiece.X)
        answer: Move = find_best_move(board=test_board, max_depth=8)
        assert answer == Move(8)
        
        

In [134]:
if __name__ == '__main__':
    unittest.main(argv=['first-arg-is-ignored'], exit=False)

...
----------------------------------------------------------------------
Ran 3 tests in 0.019s

OK


## Play Tic-Tac-Toe

We can now play tic-tac-toe against the algorithm. This time, we actually implement the move after finding the best move.

In [135]:
def get_player_move(board: Board) -> Move:
    """
    Repeatedly ask player for a move until a valid move is entered.
    """
    player_move: Move = Move(-1)
    # Prompt until move is legal
    while player_move not in board.legal_moves:
        print('\n')
        print(board)
        print('Legal Moves:', board.legal_moves, '\n')
        play: int = int(input("Enter a legal square:"))
        player_move = Move(play)
        return player_move

In [43]:
def play_tic_tac_toe() -> None:
    """
    Play tic-tac-toe against algorithm.
    """
    # Instantiate an empty board
    board: Board = TTTBoard()
        
    # Play until game is complete
    while True:
        human_move = get_player_move(board=board)
        # We update board by taking the move and reassiging a new board
        board = board.move(human_move)
        
        if board.is_win: 
            print("Human wins!")
            print(board)
            break
        elif board.is_draw:
            print("Draw!")
            print(board)
            break
            
        computer_move = find_best_move(board=board, max_depth=8)
        # Actually update the board by taking the move
        board = board.move(computer_move)
        if board.is_win:
            print("Computer wins!")
            print(board)
            break
        elif board.is_draw:
            print("Draw!")
            print(board)
            break

The computer plays perfectly (if we implemented the algorithm correctly!) so we will never be able to win. The best we can hope for is to draw, by playing ideally ourselves.

In [45]:
play_tic_tac_toe()



TTTPiece.E|TTTPiece.E|TTTPiece.E 
TTTPiece.E|TTTPiece.E|TTTPiece.E
TTTPiece.E|TTTPiece.E|TTTPiece.E
Legal Moves: [0, 1, 2, 3, 4, 5, 6, 7, 8] 



Enter a legal square: 4




TTTPiece.O|TTTPiece.E|TTTPiece.E 
TTTPiece.E|TTTPiece.X|TTTPiece.E
TTTPiece.E|TTTPiece.E|TTTPiece.E
Legal Moves: [1, 2, 3, 5, 6, 7, 8] 



Enter a legal square: 3




TTTPiece.O|TTTPiece.E|TTTPiece.E 
TTTPiece.X|TTTPiece.X|TTTPiece.O
TTTPiece.E|TTTPiece.E|TTTPiece.E
Legal Moves: [1, 2, 6, 7, 8] 



...



TTTPiece.O|TTTPiece.X|TTTPiece.E 
TTTPiece.X|TTTPiece.X|TTTPiece.O
TTTPiece.E|TTTPiece.O|TTTPiece.E
Legal Moves: [2, 6, 8] 



Enter a legal square: 8




TTTPiece.O|TTTPiece.X|TTTPiece.O 
TTTPiece.X|TTTPiece.X|TTTPiece.O
TTTPiece.E|TTTPiece.O|TTTPiece.X
Legal Moves: [6] 



Enter a legal square: 6


Draw!
TTTPiece.O|TTTPiece.X|TTTPiece.O 
TTTPiece.X|TTTPiece.X|TTTPiece.O
TTTPiece.X|TTTPiece.O|TTTPiece.X


The artificial intelligence must be coded correctly because the best outcome is a draw! This is possible because we can evaluate the complete decision tree from board position due to the limited size of the environment.

## Connect Four

There are __7__ columns with __6__ rows. The only choice is which column to drop the game piece in with a legal move in any column that is not full. The game is complete when either player has achieved 4 pieces in a column, row, or along a diagonal.
A draw occurs when all positions in the board are filled with neither player triumphant. 

In [117]:
class C4Piece(Piece, Enum):
    B = "B"
    R = "R"
    E = " " # Stand in for empty
    
    # A property is a function that behaves as an attribute
    @property
    def opposite(self) -> C4Piece:
        if self == C4Piece.B:
            return C4Piece.R
        elif self == C4Piece.R:
            return C4Piece.B
        # Opposite of empty is still empty
        else:
            return C4Piece.E
        
    def __str__(self) -> str:
        return self.value

## Potential Winning Segments in ConnectFour

In [118]:
from typing import Tuple

def generate_segments(num_columns: int, num_rows: int, segment_length: int) -> List[List[Tuple[int, int]]]:
    """
    Create all the possible winning segments to check if game
    has been won.
    """
    segments: List[List[Tuple[int, int]]] = []
        
    # Generate vertical segments
    for c in range(num_columns):
        for r in range(num_rows - segment_length + 1):
            segment: List[Tuple[int, int]] = []
            for t in range(segment_length):
                segment.append((c, r + t))
                    
            segments.append(segment)
                
    # Generate horizontal segments
    for c in range(num_columns - segment_length + 1):
        for r in range(num_rows):
            segment: List[Tuple[int, int]] = []
            for t in range(segment_length):
                segment.append((c + t, r))
                
            segments.append(segment)
            
    # Generate bottom left to upper right segments
    for c in range(num_columns - segment_length + 1):
        for r in range(num_rows - segment_length + 1):
            segment: List[Tuple[int, int]] = []
            for t in range(segment_length):
                segment.append((c + t, r + t))
            
            segments.append(segment)
            
    # Generate top left to bottom right segments
    for c in range(num_columns - segment_length + 1):
        for r in range(segment_length - 1, num_rows):
            segment: List[Tuple[int, int]] = []
            for t in range(segment_length):
                segment.append((c + t, r - t))
            
            segments.append(segment)
            
    return segments

generate_segments(num_columns=2, num_rows=2, segment_length=2)

[[(0, 0), (0, 1)],
 [(1, 0), (1, 1)],
 [(0, 0), (1, 0)],
 [(0, 1), (1, 1)],
 [(0, 0), (1, 1)],
 [(0, 1), (1, 0)]]

`segments` is a list of lists that show all the winning positions for a given color. If a color is filling any of the segments entirely, then it has won. We need a way to quickly check all of the winning segments for a given size board (and segment length), so when we create the `C4Board`, we store the winning positions with the board.

In [119]:
class C4Board(Board):
    NUM_ROWS: int = 6
    NUM_COLUMNS: int = 7
    SEGMENT_LENGTH: int = 4
    SEGMENTS: List[List[Tuple[int, int]]] = generate_segments(NUM_COLUMNS, NUM_ROWS, SEGMENT_LENGTH)
    
len(C4Board.SEGMENTS)

69

__Pieces are indexed by (Column, Row) because the board is a group of columns each of which has many rows.__

There are 69 winning segments (position sequences) for a 7 column, 6 row board with required segment length of 4.

The C4Board class has an internal Column class. This means we should think about the Connect Four class as a group of 7 columns.

In [120]:
class C4Board(Board):
    NUM_ROWS: int = 6
    NUM_COLUMNS: int = 7
    SEGMENT_LENGTH: int = 4
    SEGMENTS: List[List[Tuple[int, int]]] = generate_segments(
        NUM_COLUMNS, NUM_ROWS, SEGMENT_LENGTH
    )

    class Column:
        """
        Holds a column of the connect four board. Conceptually, we can think of the C4 Board as a group
        of seven columns.
        """

        def __init__(self) -> None:
            self._container: List[C4Piece] = []

        @property
        def full(self) -> bool:
            return len(self._container) == C4Board.NUM_ROWS

        def push(self, item: C4Piece) -> None:
            """
            Add a piece to the column if not full.
            """
            if self.full:
                raise OverflowError("Cannot push piece to already full column.")
            self._container.append(item)

        def __getitem__(self, index: int) -> C4Piece:
            """
            Implement the __getitem__ method for a column object.
            """
            if index > len(self._container) - 1:
                return C4Piece.E
            return self._container[index]

        def __repr__(self) -> str:
            return repr(self._container)

        def copy(self) -> C4Board.Column:
            temp: C4Board.Column = C4Board.Column()
            temp._container = self._container.copy()
            return temp

    def __init__(
        self, position: Optional[List[C4Board.Column]] = None, turn: C4Piece = C4Piece.B
    ) -> None:
        if position is None:
            # Representation is a list of Column objects
            self.position: List[C4Board.Column] = [
                C4Board.Column() for _ in range(C4Board.NUM_COLUMNS)
            ]
        else:
            self.position = position
        self._turn: C4Piece = turn

    # Every board must have a turn property (really an attribute but we can't require an abstract
    # base class to have a specific attribute)
    @property
    def turn(self) -> Piece:
        return self._turn

    def move(self, location: Move) -> Board:
        """
        Fill one piece in one column.
        """
        # Copy the entire board
        temp_position: List[C4Board.Column] = self.position.copy()
        # Copy each individual column
        for c in range(C4Board.NUM_COLUMNS):
            temp_position[c] = self.position[c].copy()
        # Drop the piece in the column
        temp_position[location].push(self._turn)
        # Return a new board (instead of modfying the original)
        return C4Board(position=temp_position, turn=self._turn.opposite)

    @property
    def legal_moves(self) -> List[Move]:
        """
        Return list of columns in which a move can be made. This is all columns
        which are not full.
        """
        return [
            Move(c) for c in range(C4Board.NUM_COLUMNS) if not self.position[c].full
        ]

    def _count_segment(self, segment: List[Tuple[int, int]]) -> Tuple[int, int]:
        """
        Count the number of black and red pieces in a segment. A segment is a list
        of tuples of (column, row) integers.
        """
        black_count: int = 0
        red_count: int = 0

        # Check all the positions in a segment
        for column, row in segment:
            if self.position[column][row] == C4Piece.B:
                black_count += 1
            elif self.position[column][row] == C4Piece.R:
                red_count += 1

        return black_count, red_count

    @property
    def is_win(self) -> bool:
        """
        Check if either player has won. We do this by counting the pieces in each segment. 
        If there are SEGMENT_LENGTH pieces of the same color in any segment, that player has won.
        """
        for segment in C4Board.SEGMENTS:
            # Count pieces in each segment
            black_count, red_count = self._count_segment(segment=segment)
            # Check if segments are full of any one color
            if (
                black_count == C4Board.SEGMENT_LENGTH
                or red_count == C4Board.SEGMENT_LENGTH
            ):
                return True
        return False

    # is_draw property is implement in Board's implementation

    def _evaluate_segment(self, segment: List[Tuple[int, int]], player: Piece) -> float:
        """
        Find the value of a segment to a player.
        
        Mixed segment = 0
        2 Pieces: score = 1
        3 Pieces: score = 100
        4 Pieces: score = 1_000_000
        
        If pieces are for _other_ player, score is negated.
        """
        black_count, red_count = self._count_segment(segment=segment)
        if red_count > 0 and black_count > 0:
            # Mixed segments are neutral score
            return 0

        count: int = max(black_count, red_count)
        score: float = 0

        if count == 2:
            score = 1
        # Getting close
        elif count == 3:
            score = 100
        # Win
        elif count == 4:
            score = 1_000_000

        color: C4Piece = C4Piece.B
        if red_count > black_count:
            color = C4Piece.R

        if player != color:
            # Negate score for opposite player
            return -1 * score
        return score

    def evaluate(self, player: Piece) -> int:
        """
        Evaluate total sum of positions for player.
        """
        total: float = 0
        # Find the sum of each segment for a player
        for segment in C4Board.SEGMENTS:
            total += self._evaluate_segment(segment=segment, player=player)
        return total

    def __repr__(self) -> str:
        display: str = ""
        for r in reversed(range(C4Board.NUM_ROWS)):
            display += "|"
            for c in range(C4Board.NUM_COLUMNS):
                display += f"{self.position[c][r]}" + "|"
            display += "\n"
        return display

In [121]:
l = [1, 2, 3, 4]
l.__getitem__(3)
l[3]

4

4

In [122]:
t = l.copy()
t[1] = 10
l

[1, 2, 3, 4]

## Play ConnectFour

The `minimax` and `find_best_move` functions can remain the same from tic-tac-toe. The only difference when we play is that the artificial intelligence will only evaluate the decision tree to a depth of 3. This allows for a reasonably evaluation time of moves by the computer.

In [129]:
def get_player_move(board: Board) -> Move:
    """
    Repeatedly ask player for move until a legal move is entered.
    """
    player_move: Move = Move(-1)
        
    while player_move not in board.legal_moves:
        print(board)
        print('Legal Moves', board.legal_moves, '\n')
        play: int = int(input('Enter a legal column to play (0-6):'))
        player_move = Move(play)
    
    return player_move

In [139]:
def connect_four(max_depth: int = 3, algorithm: Callable = minimax):
    """
    Play the connect four game against the algorithm using MiniMax search.
    """
    
    board: Board = C4Board()
    
    while True:
        human_move = get_player_move(board=board)

        # We update board by taking the move and reassiging a new board
        board = board.move(human_move)

        if board.is_win: 
            print("Human wins!")
            print(board)
            break
        elif board.is_draw:
            print("Draw!")
            print(board)
            break

        computer_move = find_best_move(board=board, max_depth=max_depth, algorithm=algorithm)
        print(f"Computer move: {computer_move}")
        # Actually update the board by taking the move
        board = board.move(computer_move)

        if board.is_win:
            print("Computer wins!")
            print(board)
            break
        elif board.is_draw:
            print("Draw!")
            print(board)
            break

In [131]:
connect_four(2)

| | | | | | | |
| | | | | | | |
| | | | | | | |
| | | | | | | |
| | | | | | | |
| | | | | | | |

Legal Moves [0, 1, 2, 3, 4, 5, 6] 



Enter a legal column to play (0-6): 3


Computer move: 1
| | | | | | | |
| | | | | | | |
| | | | | | | |
| | | | | | | |
| | | | | | | |
| |R| |B| | | |

Legal Moves [0, 1, 2, 3, 4, 5, 6] 



Enter a legal column to play (0-6): 3


Computer move: 3
| | | | | | | |
| | | | | | | |
| | | | | | | |
| | | |R| | | |
| | | |B| | | |
| |R| |B| | | |

Legal Moves [0, 1, 2, 3, 4, 5, 6] 



...

Computer move: 5
| | | | | | | |
| | | | | | | |
| | | |B| | | |
| | | |R| |R| |
| | |R|B|R|B| |
| |R|B|B|R|B| |

Legal Moves [0, 1, 2, 3, 4, 5, 6] 



Enter a legal column to play (0-6): 3


Computer move: 4
| | | | | | | |
| | | |B| | | |
| | | |B| | | |
| | | |R|R|R| |
| | |R|B|R|B| |
| |R|B|B|R|B| |

Legal Moves [0, 1, 2, 3, 4, 5, 6] 



Enter a legal column to play (0-6): 4


Computer move: 2
Computer wins!
| | | | | | | |
| | | |B| | | |
| | | |B|B| | |
| | |R|R|R|R| |
| | |R|B|R|B| |
| |R|B|B|R|B| |



The computer is good! We can increase the capabilities by increasing the maximum depth at the cost of increased computation time. There are also methods that can prune the decision tree to prevent the computer from evaluating branches that we know will not pay off - result in a better move.


## Alpha-Beta Pruning

To decrease the search space, we eliminate branches that we know will not yield improvements over branches already searched.

In [141]:
def alpha_beta(board: Board, maximizing: bool, original_player: Piece, 
               max_depth: int = 8, 
               alpha: float = float('-inf'), 
               beta: float = float('inf')) -> float:
    """
    Implements alpha_beta search. Stop searching a tree if beta <= alpha because we will never choose that branch.
    """
    # Base case
    if board.is_win or board.is_draw or max_depth == 0:
        # Find and return value of current position for player for whom we are evaluating
        return board.evaluate(original_player)
    
    if maximizing:
        # Find value of each move
        for move in board.legal_moves:
            result: float = alpha_beta(board = board.move(move), maximizing=False, original_player=original_player,
                                      max_depth=max_depth - 1, alpha = alpha, beta=beta)
            # Want maximum value if maximizing
            alpha = max(alpha, result)
            if beta <= alpha:
                break # Do not explore this branch further
                
        return alpha
    
    else:
        for move in board.legal_moves:
            result: float = alpha_beta(board = board.move(move), maximizing=True, original_player=original_player,
                                      max_depth=max_depth - 1, alpha = alpha, beta=beta)
                
            # Want minimum value if minimizing
            beta = min(beta, result)
            
            if beta <= alpha:
                break # Do not explore this branch further
                
        return beta                

## Play with Alpha-Beta Pruning

In [142]:
connect_four(max_depth=5, algorithm=alpha_beta)



| | | | | | | |
| | | | | | | |
| | | | | | | |
| | | | | | | |
| | | | | | | |
| | | | | | | |

Legal Moves: [0, 1, 2, 3, 4, 5, 6] 



Enter a legal square: 4


Computer move: 4


| | | | | | | |
| | | | | | | |
| | | | | | | |
| | | | | | | |
| | | | |R| | |
| | | | |B| | |

Legal Moves: [0, 1, 2, 3, 4, 5, 6] 



Enter a legal square: 2


Computer move: 3


| | | | | | | |
| | | | | | | |
| | | | | | | |
| | | | | | | |
| | | | |R| | |
| | |B|R|B| | |

Legal Moves: [0, 1, 2, 3, 4, 5, 6] 



...

Computer move: 0


| | | | | | | |
| | | | |R| | |
|R| |R|R|R| | |
|B| |R|B|B| | |
|B| |B|R|R| | |
|B| |B|R|B|B| |

Legal Moves: [0, 1, 2, 3, 4, 5, 6] 



Enter a legal square: 0


Computer move: 2


| | | | | | | |
|B| |R| |R| | |
|R| |R|R|R| | |
|B| |R|B|B| | |
|B| |B|R|R| | |
|B| |B|R|B|B| |

Legal Moves: [0, 1, 2, 3, 4, 5, 6] 



Enter a legal square: 0


Computer move: 2
Computer wins!
|B| |R| | | | |
|B| |R| |R| | |
|R| |R|R|R| | |
|B| |R|B|B| | |
|B| |B|R|R| | |
|B| |B|R|B|B| |



## Improvements to MiniMax

There are other improvements to the MiniMax algorithm besides alpha-beta pruning of the search tree allowing for deeper searches.

### Iterative Deepening

Run search function for max depth of 1, then max depth of 2, then max depth of 3 and so on until a specified time limit is reached. Result from the last completed depth is returned.

__Iterative deepening is a search for a specified period of time instead of a specified depth. This means at the beginning, worse moves may be made because the search tree is not explored as far. Over time however, the quality of the moves should increase as the same amount of time allows for deeper searches of the possible future trees.__

### Quiescence Search

Seek out and explore the more "interesting" branches, those that yield a larger change in the evaluation. The idea is that we want to look for larger advantages rather than incremental improvements in position.


### Main Improvements to MiniMax Search

* Search further in the decision tree (go deeper)
* Tune the evaluation function to return accurate measures of the value of a position (hence a move from a position).
    * Use heuristics to evaluate positions. These are rules that can inform - or speed up - the evaluation of a position
    
    
Searching for longer usually results in better positions. Evaluating positions more quickly allows us to evaluate more positions but might mean a sacrifice in quality. 

## Applications of Adverserial Search

Most chess engines still use a version of MiniMax with AlphaBeta pruning. It is still an effective technique in games with perfect knowledge. 


The higher the branching factor - number of possible moves from each position - the less effective MiniMax will be because it has to search more branching positions (this assumes a fixed max depth or time limit of search) and other methods - such as reinforcement learning have proven to be more effective.

Adverserial search is effective for __modeling economic and political situations__ as well as experiments in game theory. Alpha Beta pruning can be used with any version of MiniMax to limit the search space, which then lets us increase the search depth and find better moves in the same amount of time. 

Pure MiniMax is not that effective, but it can be improved with additional methods. Adverserial search seems simple, but is surprisingly effective at solving problems that can be expressed as a competition between states.